In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D,\
    Dense, Layer, Reshape, InputLayer, Flatten, Input, MaxPooling2D
from alibi_detect.od import OutlierAE
from alibi_detect.utils.visualize import plot_instance_score, plot_feature_outlier_image
from tqdm import tqdm

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
def img_to_np(path):  
    img_array = []
    fpaths = glob.glob(path)
    for fname in fpaths:
        img = Image.open(fname).convert("RGB")
        img = img.resize((32,32))
        img_array.append(np.asarray(img))
    images = np.array(img_array)
    return images

train, valid = train_test_split(img_to_np("D:\\cybord\\img\\train\\*.jpg"),
                                test_size = 0.2)
test = img_to_np("D:\\cybord\\img\\test\\*.jpg")

train = train.astype('float32') / 255.
valid = valid.astype('float32') / 255.
test = test.astype('float32') / 255.

print(train.shape, valid.shape, test.shape)

(3200, 32, 32, 3) (800, 32, 32, 3) (3187, 32, 32, 3)


In [3]:
encoding_dim = 1024

encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(32, 32, 3)),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ])

decoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(encoding_dim,)),
      Dense(4*4*128),
      Reshape(target_shape=(4, 4, 128)),
      Conv2DTranspose(256, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
  ])

# initialize outlier detector
od = OutlierAE(threshold=0.015,  # threshold for outlier score
                encoder_net=encoder_net,  # can also pass AE model instead
                decoder_net=decoder_net,  # of separate encoder and decoder
                )
# train
od.fit(train, epochs=30, verbose=True)


50/50 [=] - 1s 18ms/step - loss: 4.0036e-04
